In [1]:
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import gc
tqdm.pandas()

In [2]:
candidate_items = pd.read_csv("Dataset/candidate_items.csv")
item_features = pd.read_csv("Dataset/item_features.csv")
train_purchases = pd.read_csv("Dataset/train_purchases.csv")
train_sessions = pd.read_csv("Dataset/train_sessions.csv")
test_final_sessions = pd.read_csv("Dataset/test_final_sessions.csv")
test_leaderboard_sessions = pd.read_csv("Dataset/test_leaderboard_sessions.csv")
session_attributes_train = pd.read_csv("Dataset/session_attributes_train.csv")
session_attributes_leaderboard = pd.read_csv("Dataset/session_attributes_leaderboard.csv")
session_attributes_final = pd.read_csv("Dataset/session_attributes_final.csv")
item_attributes = pd.read_csv("Dataset/item_attributes.csv")

In [3]:
item_attributes

,item_id,0,1,2,3,4,5,6,7,8,...,spring_purchases,summer_purchases,autumn_purchases,seasonality_tendency_purchases,winter_tendency_purchases,spring_tendency_purchases,summer_tendency_purchases,autumn_tendency_purchases,total_view,total_purchases
0,2,1,1,1,1,1,1,1,1,1,...,0,0,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.0,0.0
1,3,1,0,0,0,1,0,0,0,0,...,0,0,5,0.584263,0.736842,0.000000,0.000000,0.263158,157.0,19.0
2,4,1,0,0,0,1,0,0,0,1,...,80,0,0,1.000000,0.000000,1.000000,0.000000,0.000000,375.0,80.0
3,7,0,0,0,0,1,0,0,0,0,...,0,0,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,1.0
4,8,1,0,1,0,1,0,0,0,1,...,10,0,0,0.510066,0.583333,0.416667,0.000000,0.000000,127.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,1,0,0,0,1,0,0,0,0,...,3,50,1,0.779479,0.000000,0.055556,0.925926,0.018519,316.0,54.0
23687,28140,1,0,1,0,0,0,0,0,0,...,0,0,67,0.755110,0.106667,0.000000,0.000000,0.893333,449.0,75.0
23688,28141,1,0,0,0,1,0,0,0,0,...,3,0,19,0.343868,0.435897,0.076923,0.000000,0.487179,152.0,39.0
23689,28142,0,0,0,0,1,1,0,0,0,...,0,0,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.0,0.0


In [6]:
session_attributes_train

,session_id,session_length,session_length_distinct,day_of_week,month,year,session_delta_time,session_mean_item_time,num_never_seen_items,num_never_purchased_items
0,3,2,1,4,12,2020,312,156.000000,0,0
1,13,1,1,4,3,2020,0,0.000000,0,1
2,18,3,3,2,8,2020,163,54.333333,0,0
3,18,3,3,2,8,2020,163,54.333333,0,0
4,18,3,3,2,8,2020,163,54.333333,0,0
...,...,...,...,...,...,...,...,...,...,...
4108845,4440001,19,18,4,10,2020,25091,1320.578947,0,19
4108846,4440001,19,18,4,10,2020,25091,1320.578947,0,19
4108847,4440001,19,18,4,10,2020,25091,1320.578947,0,19
4108848,4440001,19,18,4,10,2020,25091,1320.578947,0,19


In [5]:
item_features['category_value'] = item_features['feature_category_id'].astype(str) + '-' + item_features[
        'feature_value_id'].astype(str)
item_features

,item_id,feature_category_id,feature_value_id,category_value
0,2,56,365,56-365
1,2,62,801,62-801
2,2,68,351,68-351
3,2,33,802,33-802
4,2,72,75,72-75
...,...,...,...,...
471746,28143,68,351,68-351
471747,28143,55,390,55-390
471748,28143,11,109,11-109
471749,28143,73,91,73-91


In [6]:
category_value_keys = list(item_features['category_value'].unique())  # list of pairs category-value
category_value_values = [v for v in range(len(category_value_keys))]  # new index -- perché non fare arange?
category_value_dict = dict(zip(category_value_keys, category_value_values))
item_features['category_value'] = item_features['category_value'].map(category_value_dict)
item_features

,item_id,feature_category_id,feature_value_id,category_value
0,2,56,365,0
1,2,62,801,1
2,2,68,351,2
3,2,33,802,3
4,2,72,75,4
...,...,...,...,...
471746,28143,68,351,2
471747,28143,55,390,545
471748,28143,11,109,62
471749,28143,73,91,168


In [7]:
train_purchases_merged = train_purchases.merge(right=item_features, left_on='item_id', right_on='item_id')
train_purchases_merged

,session_id,item_id,date,feature_category_id,feature_value_id,category_value
0,3,15085,2020-12-18 21:26:47.986,56,365,0
1,3,15085,2020-12-18 21:26:47.986,47,512,107
2,3,15085,2020-12-18 21:26:47.986,37,45,349
3,3,15085,2020-12-18 21:26:47.986,49,465,156
4,3,15085,2020-12-18 21:26:47.986,31,768,110
...,...,...,...,...,...,...
21457302,4435589,3302,2020-02-12 10:30:58.773,67,566,298
21457303,4435589,3302,2020-02-12 10:30:58.773,66,886,321
21457304,4435589,3302,2020-02-12 10:30:58.773,61,898,105
21457305,4435589,3302,2020-02-12 10:30:58.773,31,768,110


In [8]:
base = train_purchases_merged.groupby(by="feature_category_id").count()[['session_id']]
base.reset_index(inplace=True)
base

,feature_category_id,session_id
0,1,35000
1,2,90855
2,3,456574
3,4,851535
4,5,457910
...,...,...
67,69,902120
68,70,34322
69,71,32877
70,72,902526


In [9]:
base.rename(columns={'session_id': 'count'}, inplace=True)
base

,feature_category_id,count
0,1,35000
1,2,90855
2,3,456574
3,4,851535
4,5,457910
...,...,...
67,69,902120
68,70,34322
69,71,32877
70,72,902526


In [10]:
base["count"] = base["count"]  / len(train_purchases_merged)
base

,feature_category_id,count
0,1,0.001631
1,2,0.004234
2,3,0.021278
3,4,0.039685
4,5,0.021341
...,...,...
67,69,0.042043
68,70,0.001600
69,71,0.001532
70,72,0.042061


In [11]:
train_sessions_merged = train_sessions.merge(right=item_features, left_on='item_id', right_on='item_id')
base_sessions = train_sessions_merged.groupby(by="feature_category_id").count()[['session_id']]
base_sessions.reset_index(inplace=True)
base_sessions.rename(columns={'session_id': 'count'}, inplace=True)
base_sessions["count"] = base_sessions["count"]  / len(train_sessions_merged)
base_sessions

,feature_category_id,count
0,1,0.001805
1,2,0.003228
2,3,0.022887
3,4,0.039996
4,5,0.022953
...,...,...
68,69,0.042606
69,70,0.001523
70,71,0.001179
71,72,0.042625


In [12]:
merged = base.merge(right=base_sessions, how='outer', left_on='feature_category_id', right_on='feature_category_id')
merged.rename(columns={'count_x': 'count_view', 'count_y': 'count_purchase'}, inplace=True)
merged

,feature_category_id,count_view,count_purchase
0,1,0.001631,1.805491e-03
1,2,0.004234,3.227546e-03
2,3,0.021278,2.288701e-02
3,4,0.039685,3.999629e-02
4,5,0.021341,2.295337e-02
...,...,...,...
68,70,0.001600,1.523036e-03
69,71,0.001532,1.178810e-03
70,72,0.042061,4.262454e-02
71,73,0.038021,3.710271e-02


In [13]:
merged["count_view"] = (merged["count_view"] - merged["count_view"].min()) / (merged["count_view"].max() - merged["count_view"].min())
merged["count_purchase"] = (merged["count_purchase"] - merged["count_purchase"].min()) / (merged["count_purchase"].max() - merged["count_purchase"].min())
merged

,feature_category_id,count_view,count_purchase
0,1,0.034969,0.039156
1,2,0.090826,0.069996
2,3,0.456557,0.496354
3,4,0.851530,0.867406
4,5,0.457893,0.497793
...,...,...,...
68,70,0.034291,0.033030
69,71,0.032846,0.025565
70,72,0.902523,0.924406
71,73,0.815832,0.804653


In [14]:
merged.fillna(value={"count_purchase": 0.0, "count_view": 0.0}, inplace=True)
merged

,feature_category_id,count_view,count_purchase
0,1,0.034969,0.039156
1,2,0.090826,0.069996
2,3,0.456557,0.496354
3,4,0.851530,0.867406
4,5,0.457893,0.497793
...,...,...,...
68,70,0.034291,0.033030
69,71,0.032846,0.025565
70,72,0.902523,0.924406
71,73,0.815832,0.804653


In [15]:
merged["score"] = ((merged['count_view'] <= 1e-5) & (merged['count_purchase'] <= 1e-5))
merged

,feature_category_id,count_view,count_purchase,score
0,1,0.034969,0.039156,False
1,2,0.090826,0.069996,False
2,3,0.456557,0.496354,False
3,4,0.851530,0.867406,False
4,5,0.457893,0.497793,False
...,...,...,...,...
68,70,0.034291,0.033030,False
69,71,0.032846,0.025565,False
70,72,0.902523,0.924406,False
71,73,0.815832,0.804653,False


In [16]:
merged.loc[merged['score'] == True, 'count_view'] = 0
merged.loc[merged['score'] == True, 'count_purchase'] = 0
merged

,feature_category_id,count_view,count_purchase,score
0,1,0.034969,0.039156,False
1,2,0.090826,0.069996,False
2,3,0.456557,0.496354,False
3,4,0.851530,0.867406,False
4,5,0.457893,0.497793,False
...,...,...,...,...
68,70,0.034291,0.033030,False
69,71,0.032846,0.025565,False
70,72,0.902523,0.924406,False
71,73,0.815832,0.804653,False


In [17]:
item_features = item_features.merge(right=merged, left_on='feature_category_id', right_on='feature_category_id', how='outer')
item_features

,item_id,feature_category_id,feature_value_id,category_value,count_view,count_purchase,score
0,2,56,365,0,1.000000,1.000000,False
1,3,56,365,0,1.000000,1.000000,False
2,4,56,365,0,1.000000,1.000000,False
3,7,56,153,45,1.000000,1.000000,False
4,8,56,365,0,1.000000,1.000000,False
...,...,...,...,...,...,...,...
471746,26817,13,698,439,0.006418,0.006589,False
471747,26920,13,698,439,0.006418,0.006589,False
471748,26967,13,698,439,0.006418,0.006589,False
471749,27387,13,252,500,0.006418,0.006589,False


In [18]:
item_features.fillna(value={"count_purchase": 0.0, "count_view": 0.0, "score": True}, inplace=True)

In [19]:
item_features.drop(columns=["score",	"feature_value_id",	"category_value"], inplace=True)
item_features.rename(columns={"count_view": "count_view_feature_category_id", "count_purchase": "count_purchase_feature_category_id"}, inplace=True)
item_features

,item_id,feature_category_id,count_view_feature_category_id,count_purchase_feature_category_id
0,2,56,1.000000,1.000000
1,3,56,1.000000,1.000000
2,4,56,1.000000,1.000000
3,7,56,1.000000,1.000000
4,8,56,1.000000,1.000000
...,...,...,...,...
471746,26817,13,0.006418,0.006589
471747,26920,13,0.006418,0.006589
471748,26967,13,0.006418,0.006589
471749,27387,13,0.006418,0.006589


In [20]:
fresh = pd.read_csv("Dataset/item_features.csv")

In [22]:
a = fresh.groupby(by=['item_id', 'feature_category_id']).count().reset_index()
a.loc[a.feature_value_id > 1]

,item_id,feature_category_id,feature_value_id
129,10,30,2
144,11,30,3
238,18,30,4
402,30,4,3
478,35,30,2
...,...,...,...
461080,28125,30,3
461095,28126,30,3
461191,28133,28,2
461193,28133,30,3


In [29]:
item_features[(item_features.count_view_feature_category_id == 0.0) & (item_features.count_purchase_feature_category_id == 0.0)].feature_category_id

471750    27
Name: feature_category_id, dtype: int64